# TensorFlow Parte III - Variables

Con variables podemos representar datos que cambiarán durante el cálculo. Usamos la clase `tf.Variable` que representa un tensor con valores que se puede modificar.

In [1]:
import tensorflow as tf

2023-09-28 12:59:34.514308: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 12:59:35.352380: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Crear una variable

In [2]:
mi_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
mi_variable = tf.Variable(mi_tensor)

2023-09-28 12:59:55.444779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-28 12:59:55.465837: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Variables pueden tener cualquier tipo

In [3]:
variable_bool = tf.Variable([False, False, False, True])
variable_compleja = tf.Variable([5+4j, 6+1j])

Variables en TensorFlow son muy similares a tensores. Tienen un `dtype`, una forma y se puede exportar a NumPy.

In [4]:
mi_variable.shape

TensorShape([2, 2])

In [5]:
mi_variable.dtype

tf.float32

In [6]:
mi_variable.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

La mayoría de las operaciones de tensores funcionan en variables, pero no se puede reformar una variable.

In [7]:
mi_variable

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

In [8]:
tf.convert_to_tensor(mi_variable)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

In [9]:
tf.math.argmax(mi_variable)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 1])>

Haciendo un reshape, creamos un nuevo tensor, no cambiamos la variable.

In [10]:
tf.reshape(mi_variable, [1, 4])

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[1., 2., 3., 4.]], dtype=float32)>

Las variables están respaldadoas por tensores. Se puede reasignar el tensor usando `tf.Variable.assign`. Normalmente esta operación ocupa la memoria ya existente del tensor.

In [11]:
a = tf.Variable([2.0, 3.0])

Abajo cambiamos los valores, manteniendo el tipo.

In [12]:
a.assign([1, 2])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>

En el ejemplo abajo sería necesario cambiar el tamaño de la variable, que no está permitido:

In [ ]:
try:
    a.assign([1.0, 2.0, 3.0])
except Exception as e:
    print(f"{type(e).__name__}: {e}")

Si ocupas una variable como un tensor en operaciones, típicamente vas a operar en el tensor de fondo.

Creando variables nuevas de variables existentes duplica los tensores de fondo. Dos variables no comparten la misma memoria.

In [ ]:
a = tf.Variable([2.0, 3.0])

In [ ]:
b = tf.Variable(a)

In [ ]:
a.assign([5, 6])

In [ ]:
print(a.numpy())

In [ ]:
print(b.numpy())

Hay otras versiones de `assign`:

In [ ]:
print(a.assign_add([2,3]).numpy())

In [ ]:
print(a.assign_sub([7,9]).numpy())

## Ciclos de vida, nombramiento y seguimiento

En TensorFlow de Python, una instancia de `tf.Variable` tiene el mismo "ciclo de vida" de cualquier objeto de Python. Si no hay referencias a la variable, será desagniada automáticamente.

También se puede nombrar variables, que ayuda en el seguimiento y el proceso de *debugging*. Se puede nombrar dos variables igual.

In [ ]:
a = tf.Variable(mi_tensor, name="Pedro")

In [ ]:
b = tf.Variable(mi_tensor + 1, name="Pedro")

In [ ]:
print(a == b)

In [ ]:
a.name

Los nombres de variables están mantenidos cuando se guarda y carga los modelos. Por defecto, las variables en un modelo adquieren nombres únicos automáticamente.

A veces hay variables que no queremos incluir en el proceso de entrenamiento, es decir, no hace falta su derivación. Se puede controlar eso con la propiedad `trainable`:

In [ ]:
contador = tf.Variable(1, trainable=False)